In [25]:
import re
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import keras # even though we import this, we are not using this import to access preprocessing functionalities.
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
import math
import nltk

In [10]:
df = pd.read_csv('/content/sentiment_train.csv')
df.head()

,text,lable
0,"I`d have responded, if I were going",neutral
1,Sooo SAD,negative
2,bullying me,negative
3,leave me alone,negative
4,"Sons of ****,",negative


In [11]:
label_counts = df['lable'].value_counts()
print(label_counts)

lable
neutral     11118
positive     8582
negative     7781
Name: count, dtype: int64


In [12]:
from sklearn.utils import resample

# Separate majority and minority classes
df_majority = df[df['lable'] == 'neutral']
df_minority = df[df['lable'] == 'negative']
df_middle = df[df['lable'] == 'positive']

# Downsample majority class
df_majority_downsampled = resample(df_majority,
                                 replace=False,    # sample without replacement
                                 n_samples=len(df_minority),     # to match minority class
                                 random_state=123) # reproducible results

# Combine minority class with downsampled majority class
df_downsampled = pd.concat([df_majority_downsampled, df_minority, df_middle])

# Display new class counts
print(df_downsampled['lable'].value_counts())

lable
positive    8582
neutral     7781
negative    7781
Name: count, dtype: int64


In [13]:
df_downsampled['text'] = df_downsampled['text'].astype(str).apply(lambda x: x.lower())
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
df_downsampled['text'] = df_downsampled['text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [14]:
import nltk
nltk.download('wordnet')
w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
lemmatizer = nltk.stem.WordNetLemmatizer()
def lemmatize_text(text):
    st = ""
    for w in w_tokenizer.tokenize(text):
        st = st + lemmatizer.lemmatize(w) + " "
    return st
df_downsampled['text'] = df_downsampled.text.apply(lemmatize_text)
df_downsampled

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


,text,lable
18506,_devil1 big brother quiz? what?! i`m busy rock...,neutral
24,see wat mean bout foll0w friidays... it`s call...,neutral
1656,hence smiley face. ha ha ha ha. lol yeah i`m h...,neutral
2258,found one know using twitter.,neutral
4865,try. don`t care mum say,neutral
...,...,...
27467,welcome,positive
27474,feel good ab,positive
27475,enjoy,positive
27478,yay good you.,positive


In [15]:
reviews = df_downsampled['text'].values
labels = df_downsampled['lable'].values
encoder = LabelEncoder()
encoded_labels = encoder.fit_transform(labels)

In [33]:
print(encoded_labels)

[1 1 1 ... 2 2 2]


In [16]:
train_sentences, test_sentences, train_labels, test_labels = train_test_split(reviews, encoded_labels, stratify = encoded_labels)

In [17]:
#set model params
# Hyperparameters of the model
vocab_size = 3000 # choose based on statistics
oov_tok = ''
embedding_dim = 100
max_length = 200 # choose based on statistics, for example 150 to 200
padding_type='post'
trunc_type='post'
# tokenize sentences
tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(train_sentences)
word_index = tokenizer.word_index
# convert train dataset to sequence and pad sequences
train_sequences = tokenizer.texts_to_sequences(train_sentences)
train_padded = pad_sequences(train_sequences, padding='post', maxlen=max_length)
# convert Test dataset to sequence and pad sequences
test_sequences = tokenizer.texts_to_sequences(test_sentences)
test_padded = pad_sequences(test_sequences, padding='post', maxlen=max_length)

In [31]:
# model initialization
model = keras.Sequential()
model.add(Embedding(vocab_size, embedding_dim)),
model.add(SpatialDropout1D(0.2))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))
# compile model
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
# model summary
model.summary()

Model: "sequential_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_7 (Embedding)              │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ spatial_dropout1d_3                  │ ?                           │               0 │
│ (SpatialDropout1D)                   │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_7 (LSTM)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_11 (Dense)                     │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [32]:
#model training
num_epochs = 5
history = model.fit(train_padded, train_labels,
                    epochs=num_epochs, verbose=1,
                    validation_split=0.1)

Epoch 1/5
404/510 ━━━━━━━━━━━━━━━━━━━━ 38s 362ms/step - accuracy: 0.3242 - loss: 0.0000e+00

KeyboardInterrupt: 

In [34]:
accr = model.evaluate(test_padded,train_padded)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

ValueError: Data cardinality is ambiguous. Make sure all arrays contain the same number of samples.'x' sizes: 6036
'y' sizes: 18108
